In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import TruncatedSVD

import numpy as np

#path = "../../Data/"
words = []


def train_to_sentence_form(path):
    # reads the file to a format of docs[doc[[sentence1],[sentence2]],...]
    docs=[]
    num_of_sentences = []
    with open(path) as f:
        line = f.readline()
        while line:
            line = line.rstrip('\n').split(" ")
            #print(line)
            num = line.pop(0)
            #print(num)
            number = int(num[1:-1])
            num_of_sentences.append(number)
            len_of_sentence = []

            for i,k in enumerate(line):
                if k.startswith('<'):
                    k=k[1:-1]
                    len_of_sentence.append(int(k))
                    line.pop(i)
            doc_sentence=[]       
            sentence=[]
            for j in len_of_sentence:
                sentence = line[:j]
                line=line[j:]
                docs.append( sentence) 
            line = f.readline()
    print(len(docs))
    return docs,num_of_sentences     
    #docs is a list of lists where each list is a different sentence





vocabs = open("vocabs.txt").readlines()
for vocab in vocabs:
    word, _ = vocab.split(', ')
    words.append(word)

train_label = [i.strip() for i in open("train-label.dat").readlines()]
train_label = np.array(train_label)

#path="../Data/"
docs,num_sentences = train_to_sentence_form("train-data.dat")

#train_data = convert_to_word(path + "/train-data.dat")
data = []
for i in docs:
    final_L = ''
    for l in i:
        final_L = final_L + ' ' + words[int(l)]
    data.append(final_L[1:])
print(data[:12])
print(len(data))       
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data)
print(X_train_counts.size)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
svd = TruncatedSVD(n_components=20,n_iter=5, random_state=1)
mat_reduced = svd.fit_transform(X_train_tfidf)
Data_new = np.array(mat_reduced)

149925
['rubi rail helper demo more info auto complet', 'see new helper action', 'null length substr locat', 'exec messag messag pleas edit remov follow word content', 'roll stone com news song previou next page', 'good vibrat beach boy', 'smell teen spirit nirvana', 'want hold hand beatl', 'hound dog elvi', 'god know beach boy', 'walk line johnni cash', 'heaven led zeppelin']
149925
965852


In [2]:
def multilabel_confusion_matrix(predict,clustnum):
    test_label = [i.strip().split() for i in open( "test-label.dat").readlines()]

    for i in range(len(test_label)):
        item = test_label[i]
        for j in range(len(item)):
            item[j] = int(item[j])

        test_label[i] = item

    labels = []

    for item in open ("labels.txt").readlines():
        item = item.replace('\n', '')
        label_name, _ = item.split(', ')
        labels.append(label_name)

    y_true = np.array(test_label)
    y_pred = np.array(predict)

    conf_mat_dict = {}

    for label_col in range(len(labels)):
        y_true_label = y_true[:, label_col]
        y_pred_label = y_pred[:, label_col]
        conf_mat_dict[labels[label_col]] = confusion_matrix(y_pred=y_pred_label, y_true=y_true_label)
    f = open("confmatrix"+str(clustnum)+".txt", "w")
    f.close()
    for label, matrix in conf_mat_dict.items():
        print("Confusion matrix for label {}:".format(label))
        f = open("confmatrix"+str(clustnum)+".txt", "a")
        print(matrix)
        f.write("Confusion matrix for label {}:".format(label))
        np.savetxt(f,matrix)
        f.close()
    return


In [3]:
import pandas as pd
# KMeans clustering a kind of clustering.
from sklearn.cluster import KMeans
#print(train_label.size)
number_of_clusters_list = [20]
for number_of_clusters in number_of_clusters_list:
    km = KMeans(n_clusters=number_of_clusters)
    # fit the matrix
    print("fitting")
    km = km.fit(Data_new)
    initform = np.zeros([train_label.size,number_of_clusters])
    df = pd.DataFrame(initform,columns=["Cluster "+ str(i) for i in range(number_of_clusters)])
    print(km.labels_.size)
    clust_labels = km.labels_
    acc = 0
    for i,val in enumerate(num_sentences):
        for j in range(val):
            df.iloc[i][clust_labels[acc]] += 1
            acc+=1

    df.head(20)


    print("train")
    from sklearn import neighbors
    clf = neighbors.KNeighborsClassifier(n_neighbors=3 ,weights='distance',metric='minkowski',p=1).fit(df, train_label)

    test_data,test_sentences = train_to_sentence_form("test-data.dat")
    test_label = [i.strip() for i in open("test-label.dat").readlines()]
    test_label = np.array(test_label)

    data2 = []
    for i in test_data:
        final_L = ''
        for l in i:
            final_L = final_L + ' ' + words[int(l)]
        data2.append(final_L[1:])

    X_test_counts = count_vect.transform(data2)
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)
    mat_reduced = svd.transform(X_test_tfidf)
    Data_new_test = np.array(mat_reduced)
    #print("fitting")
    #km.fit(X_test_tfidf)
    #print(km.labels_.size)
    clusttest_labels = km.predict(Data_new_test)
    print(clusttest_labels)
    testform = np.zeros([test_label.size,number_of_clusters])
    df_test = pd.DataFrame(testform,columns=["Cluster "+ str(i) for i in range(number_of_clusters)])

    #clusttest_labels = km.labels_
    acctest = 0
    for i,val in enumerate(test_sentences):
        for j in range(val):
            df_test.iloc[i][clusttest_labels[acctest]] += 1
            acctest+=1

    df_test.head(20)

    pred = clf.predict(df_test)
    print(pred)
    
    predict=[]
    for p in pred:
        result = p.split()
        for r in range(len(result)):
            result[r] = int(result[r])
        predict.append(result)
    
    multilabel_confusion_matrix(predict,number_of_clusters)

fitting
149925
train
73363
[ 7 17  5 ... 18 14 17]
['1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0'
 '0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0'
 '1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0' ...
 '0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0'
 '0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0'
 '0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0']
Confusion matrix for label programming:
[[2709  297]
 [ 781  196]]
Confusion matrix for label style:
[[3614  141]
 [ 214   14]]
Confusion matrix for label reference:
[[1860  565]
 [1099  459]]
Confusion matrix for label java:
[[3410  201]
 [ 320   52]]
Confusion matrix for label web:
[[2482  451]
 [ 761  289]]
Confusion matrix for label internet:
[[3088  358]
 [ 459   78]]
Confusion matrix for label culture:
[[2758  523]
 [ 548  154]]
Confusion matrix for label design:
[[2297  607]
 [ 803  276]]
Confusion matrix for label education:
[[2630  550]
 [ 651  152]]
Confusion matrix for label language:
[[3082  418]
 [ 415   68]]
Confusion matrix for label books:
[[3046  430]
 [ 432   

In [8]:
df.to_csv("train_kmeans_svd.csv",index=False)
df_test.to_csv("test_kmeanssvd.csv",index=False)